## Classical benchmark testing

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

### Load Dataset

In [2]:
benign_path = "500-benign.npy"
attack_path = "500-attack.npy"

qX1 = np.load(benign_path)
qX2 = np.load(attack_path)

print("Loaded:", qX1.shape, qX2.shape)

Loaded: (500, 8) (500, 8)


### Use Smaller Subset

In [3]:
N = 150  
qX1 = qX1[:N]
qX2 = qX2[:N]

qX = np.vstack([qX1, qX2])
qy = np.array([-1] * len(qX1) + [1] * len(qX2))

print("Subset used:", qX.shape)

Subset used: (300, 8)


### Train / Test Split

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(
    qX, qy,
    test_size=0.3,
    random_state=42,
    stratify=qy
)

print("Split:", xtrain.shape, xtest.shape)

Split: (210, 8) (90, 8)


### Normalize

In [5]:
scaler = StandardScaler()
samples = np.append(xtrain, xtest, axis=0)

scaler.fit(samples)
xtrain = scaler.transform(xtrain)
xtest  = scaler.transform(xtest)

print("Data standardized.")

Data standardized.


### Train Classical SVM

In [6]:
svc = SVC()   # Default RBF kernel
print("Model:", svc)

svc.fit(xtrain, ytrain)

train_acc = svc.score(xtrain, ytrain)
print("🎯 TRAIN ACCURACY:", train_acc)

Model: SVC()
🎯 TRAIN ACCURACY: 0.680952380952381


### Cross Validation

In [7]:
cv_scores = cross_val_score(svc, xtrain, ytrain, cv=5)
print("✅ CV MEAN:", cv_scores.mean())

✅ CV MEAN: 0.5904761904761904


### Final Test Evaluation

In [8]:
ypred = svc.predict(xtest)

acc = np.mean(ypred == ytest)
print("✅ TEST ACCURACY:", acc)

cm = confusion_matrix(ytest, ypred)
print("\nConfusion Matrix:\n", cm)

cr = classification_report(ytest, ypred)
print("\nClassification Report:\n", cr)

✅ TEST ACCURACY: 0.6

Confusion Matrix:
 [[32 13]
 [23 22]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.58      0.71      0.64        45
           1       0.63      0.49      0.55        45

    accuracy                           0.60        90
   macro avg       0.61      0.60      0.59        90
weighted avg       0.61      0.60      0.60        90

